In [2]:
%pip install numpy scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [4]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [5]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
tfidfvect = TfidfVectorizer()

In [7]:
# en el atributo `data` accedemos al texto
print(newsgroups_train.data[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [18]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [16]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [19]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [20]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [21]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [22]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

### Vectorización y análisis de similaridad entre documentos

A continuación, se vectorizan los documentos, se seleccionan 5 documentos al azar y se mide la similaridad de cada uno con el resto del corpus. Para cada documento, se analizan los 5 documentos más similares considerando el contenido y la etiqueta de clasificación.

In [23]:
import random

# Seleccionar 5 documentos al azar del conjunto de entrenamiento
random.seed(42)
random_docs = random.sample(range(X_train.shape[0]), 5)

# Para cada documento seleccionado, calcular la similaridad coseno con el resto
doc_similarities = {}
for idx in random_docs:
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    # Excluir el propio documento (índice idx)
    most_similar = np.argsort(cossim)[::-1][1:6]
    doc_similarities[idx] = most_similar

# Mostrar los índices de los documentos seleccionados y sus 5 más similares
doc_similarities

{10476: array([ 5064,  9623, 10575, 10836,  2350]),
 1824: array([9921, 6364, 5509, 2641, 4359]),
 409: array([3444, 5799, 5905, 1764, 3364]),
 4506: array([4211, 5928, 6224, 5171, 9491]),
 4012: array([ 6599, 10644,  7478,  7308, 10792])}

In [24]:
for i, idx in enumerate(random_docs):
    print(f"\nDocumento {i+1} (índice {idx}):")
    print("Clase:", newsgroups_train.target_names[y_train[idx]])
    print("Texto:\n", newsgroups_train.data[idx][:500], "...\n")
    print("Documentos más similares:")
    for j, sim_idx in enumerate(doc_similarities[idx]):
        print(f"  Similar {j+1} (índice {sim_idx}): Clase: {newsgroups_train.target_names[y_train[sim_idx]]}")
        print("  Texto:\n ", newsgroups_train.data[sim_idx][:300], "...\n")
    print("-"*80)


Documento 1 (índice 10476):
Clase: rec.sport.hockey
Texto:
 This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an opportunity to watch this year are Wash-NYI and Chi-Stl, the latter because
I'm in the wrong time zone!).  We (in Canada) are basically swamped with 
coverage, and I wonder how many series/games are televised nationally or even
locally in the US and how much precedence they take over, say, local new ...

Documentos más similares:
  Similar 1 (índice 5064): Clase: rec.sport.hockey
  Texto:
  
I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are THE most exciting
games to watch. ...

  Similar 2 (índice 9623): Clase: talk.politics.mideast
  Texto:
  Accounts of Anti-Armenian Human Right 

### Optimización de modelos Naïve Bayes y vectorizadores

Se probarán diferentes configuraciones de vectorizadores (TF-IDF y CountVectorizer) y modelos Naïve Bayes (MultinomialNB y ComplementNB), variando parámetros como ngram_range, uso de stopwords y max_features. Se reportará el mejor f1-score macro obtenido en el conjunto de test y los parámetros asociados.

In [25]:
from sklearn.model_selection import ParameterGrid
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

vectorizer_options = [
    {'vectorizer': TfidfVectorizer, 'name': 'TF-IDF'},
    {'vectorizer': CountVectorizer, 'name': 'Count'}
]

param_grid = {
    'ngram_range': [(1,1), (1,2)],
    'stop_words': [None, 'english'],
    'max_features': [None, 5000]
}

models = [
    {'model': MultinomialNB, 'name': 'MultinomialNB'},
    {'model': ComplementNB, 'name': 'ComplementNB'}
]

best_score = 0
best_config = None

for vect_opt in vectorizer_options:
    for params in ParameterGrid(param_grid):
        vect = vect_opt['vectorizer'](
            ngram_range=params['ngram_range'],
            stop_words=params['stop_words'],
            max_features=params['max_features']
        )
        X_train_vect = vect.fit_transform(newsgroups_train.data)
        X_test_vect = vect.transform(newsgroups_test.data)
        for model_opt in models:
            clf = model_opt['model']()
            clf.fit(X_train_vect, y_train)
            y_pred = clf.predict(X_test_vect)
            score = f1_score(y_test, y_pred, average='macro')
            if score > best_score:
                best_score = score
                best_config = {
                    'vectorizer': vect_opt['name'],
                    'ngram_range': params['ngram_range'],
                    'stop_words': params['stop_words'],
                    'max_features': params['max_features'],
                    'model': model_opt['name']
                }
            print(f"Vectorizer: {vect_opt['name']}, Model: {model_opt['name']}, Params: {params}, F1-macro: {score:.4f}")

print("\nMejor configuración:")
print(best_config)
print(f"Mejor F1-score macro: {best_score:.4f}")

Vectorizer: TF-IDF, Model: MultinomialNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': None}, F1-macro: 0.5854
Vectorizer: TF-IDF, Model: ComplementNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': None}, F1-macro: 0.6930
Vectorizer: TF-IDF, Model: MultinomialNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': 'english'}, F1-macro: 0.6468
Vectorizer: TF-IDF, Model: ComplementNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': 'english'}, F1-macro: 0.6936
Vectorizer: TF-IDF, Model: MultinomialNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': 'english'}, F1-macro: 0.6468
Vectorizer: TF-IDF, Model: ComplementNB, Params: {'max_features': None, 'ngram_range': (1, 1), 'stop_words': 'english'}, F1-macro: 0.6936
Vectorizer: TF-IDF, Model: MultinomialNB, Params: {'max_features': None, 'ngram_range': (1, 2), 'stop_words': None}, F1-macro: 0.5398
Vectorizer: TF-IDF, Model: MultinomialNB, Par

### Similaridad entre palabras a partir de la matriz término-documento

Se transpone la matriz documento-término para obtener una matriz término-documento, donde cada fila representa un vector de una palabra en el espacio de los documentos. Se seleccionan 5 palabras relevantes manualmente y se calcula la similaridad coseno entre sus vectores y los del resto del vocabulario. Se muestran las 5 palabras más similares para cada una.

In [26]:
# Transponer la matriz documento-término para obtener término-documento
X_terms = X_train.transpose()

# Seleccionar 5 palabras relevantes manualmente (asegúrate de que existan en el vocabulario)
selected_words = ['car', 'god', 'computer', 'space', 'music']

# Obtener los índices de las palabras seleccionadas
token2idx = tfidfvect.vocabulary_
idx2token = {v: k for k, v in token2idx.items()}
selected_indices = [token2idx[w] for w in selected_words if w in token2idx]

for idx in selected_indices:
    word = idx2token[idx]
    # Similaridad coseno entre la palabra y todas las demás
    cossim = cosine_similarity(X_terms[idx], X_terms)[0]
    # Excluir la propia palabra
    most_similar = np.argsort(cossim)[::-1][1:6]
    print(f"\nPalabra: '{word}'")
    print("Palabras más similares:")
    for sim_idx in most_similar:
        print(f"  {idx2token[sim_idx]} (similitud: {cossim[sim_idx]:.3f})")


Palabra: 'car'
Palabras más similares:
  cars (similitud: 0.180)
  criterium (similitud: 0.177)
  civic (similitud: 0.175)
  owner (similitud: 0.169)
  dealer (similitud: 0.168)

Palabra: 'god'
Palabras más similares:
  jesus (similitud: 0.269)
  bible (similitud: 0.262)
  that (similitud: 0.256)
  existence (similitud: 0.255)
  christ (similitud: 0.251)

Palabra: 'computer'
Palabras más similares:
  decwriter (similitud: 0.156)
  harkens (similitud: 0.152)
  deluged (similitud: 0.152)
  shopper (similitud: 0.144)
  the (similitud: 0.136)

Palabra: 'space'
Palabras más similares:
  nasa (similitud: 0.330)
  seds (similitud: 0.297)
  shuttle (similitud: 0.293)
  enfant (similitud: 0.280)
  seti (similitud: 0.246)

Palabra: 'music'
Palabras más similares:
  inconveniences (similitud: 0.415)
  colect (similitud: 0.415)
  posses (similitud: 0.415)
  deaf (similitud: 0.408)
  competencies (similitud: 0.402)
